##### Here I am going to process the data and transform it in different ways to see if the predicition models give different and better results.

In [144]:
# Data processing
# ---------------------------
import pandas as pd
import numpy as np

# Dates
# ---------------------------
import datetime as dt
from datetime import datetime

# Data balancing
# ---------------------------
from imblearn.combine import SMOTETomek
from collections import Counter

# Standardization
# ---------------------------
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

In [145]:
# Loading the dataframe

df = pd.read_csv('data/train.csv')
df.head()

,order_id,local_time,country_code,store_address,payment_status,n_of_products,products_total,final_status
0,33446280,14:11:09,AR,55379,PAID,2,11.88,DeliveredStatus
1,33107339,11:47:41,GT,23487,PAID,2,5.20,DeliveredStatus
2,32960645,11:53:53,CR,62229,PAID,1,6.03,DeliveredStatus
3,32089564,20:15:21,ES,29446,PAID,6,6.37,DeliveredStatus
4,32157739,21:32:16,AR,13917,PAID,1,5.36,CanceledStatus


# Encoding

### country_code

In [146]:
dummies1 = pd.get_dummies(df['country_code'])
df2= pd.concat([df,dummies1], axis=1)
df2.drop('country_code', axis=1, inplace=True)
df2.head()

,order_id,local_time,store_address,payment_status,n_of_products,products_total,final_status,AR,BR,CI,...,KE,MA,PA,PE,PR,PT,RO,TR,UA,UY
0,33446280,14:11:09,55379,PAID,2,11.88,DeliveredStatus,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33107339,11:47:41,23487,PAID,2,5.20,DeliveredStatus,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32960645,11:53:53,62229,PAID,1,6.03,DeliveredStatus,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,32089564,20:15:21,29446,PAID,6,6.37,DeliveredStatus,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32157739,21:32:16,13917,PAID,1,5.36,CanceledStatus,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### payment_status

In [147]:
dummies2 = pd.get_dummies(df['payment_status'])
df2= pd.concat([df2, dummies2], axis=1)
df2.drop('payment_status', axis=1, inplace=True)
df2.head()

,order_id,local_time,store_address,n_of_products,products_total,final_status,AR,BR,CI,CL,...,PE,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID
0,33446280,14:11:09,55379,2,11.88,DeliveredStatus,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33107339,11:47:41,23487,2,5.20,DeliveredStatus,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,32960645,11:53:53,62229,1,6.03,DeliveredStatus,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,32089564,20:15:21,29446,6,6.37,DeliveredStatus,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,32157739,21:32:16,13917,1,5.36,CanceledStatus,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### final_status

In [148]:
map = {'CanceledStatus':0, 'DeliveredStatus':1}

df2['final_status'] = df2['final_status'].map(map)
df2.head()

,order_id,local_time,store_address,n_of_products,products_total,final_status,AR,BR,CI,CL,...,PE,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID
0,33446280,14:11:09,55379,2,11.88,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33107339,11:47:41,23487,2,5.20,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,32960645,11:53:53,62229,1,6.03,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,32089564,20:15:21,29446,6,6.37,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,32157739,21:32:16,13917,1,5.36,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### local_time

Since this column can't be encoded and it is still not numeric, I am going to transform the time into seconds


In [149]:
# Let's create a function to insert in an apply method
def convert_to_seconds (x):
    pt = datetime.strptime(x,'%H:%M:%S')
    total_seconds = pt.second + pt.minute*60 + pt.hour*3600
    return total_seconds

In [150]:
df2['local_time'] = df2['local_time'].apply(convert_to_seconds)
df2.head()

,order_id,local_time,store_address,n_of_products,products_total,final_status,AR,BR,CI,CL,...,PE,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID
0,33446280,51069,55379,2,11.88,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33107339,42461,23487,2,5.20,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,32960645,42833,62229,1,6.03,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,32089564,72921,29446,6,6.37,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,32157739,77536,13917,1,5.36,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [151]:
# Saving into a new csv
df2.to_csv('data/train_encoded.csv')

# Balancing the response variable with SMOTETomek

In [152]:
# Dividing in predictors and response, the variable we want to balance
y = df2['final_status']
X = df2.drop('final_status', axis=1)

In [153]:
# Initiating the method
sm = SMOTETomek(random_state=1)

# Adjusting the model
Xres, yres = sm.fit_resample(X, y)

In [154]:
# If we compare the value counts we can see the method has worked
Counter(y)

Counter({1: 48498, 0: 5832})

In [155]:
Counter(yres)

Counter({1: 44954, 0: 44954})

In [156]:
# If we concatenate the resamples we obtain a new dataframe of much bigger size
df3 = pd.concat([Xres, yres], axis=1)
df3.head()

,order_id,local_time,store_address,n_of_products,products_total,AR,BR,CI,CL,CR,...,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID,final_status
0,33446280,51069,55379,2,11.88,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,33107339,42461,23487,2,5.20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,32960645,42833,62229,1,6.03,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
3,32157739,77536,13917,1,5.36,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,33214981,81292,11694,2,4.85,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [157]:
# Checking the value counts are still correct
df3.final_status.value_counts()

1    44954
0    44954
Name: final_status, dtype: int64

In [158]:
# Checking the new size of the dataframe
df3.shape

(89908, 32)

In [159]:
# I'm going to change the dtypes of order and store id to 'object', so that they don't affect quantitatively to the model.
df3[['order_id', 'store_address']] = df3[['order_id', 'store_address']].astype('object')
df3.dtypes

order_id           object
local_time          int64
store_address      object
n_of_products       int64
products_total    float64
AR                  uint8
BR                  uint8
CI                  uint8
CL                  uint8
CR                  uint8
DO                  uint8
EC                  uint8
EG                  uint8
ES                  uint8
FR                  uint8
GE                  uint8
GT                  uint8
IT                  uint8
KE                  uint8
MA                  uint8
PA                  uint8
PE                  uint8
PR                  uint8
PT                  uint8
RO                  uint8
TR                  uint8
UA                  uint8
UY                  uint8
DELAYED             uint8
NOT_PAID            uint8
PAID                uint8
final_status        int64
dtype: object

In [160]:
# Saving into a new csv
df3.to_csv('data/train_encoded_balanced.csv')

# Standardization / Normalization of numerical variables

In [161]:
# Originally, the columns with integer dtypes are these. 
# We have to include the date transformed into seconds and we should remove the order and store id.
df.select_dtypes(np.number)

,order_id,store_address,n_of_products,products_total
0,33446280,55379,2,11.88
1,33107339,23487,2,5.20
2,32960645,62229,1,6.03
3,32089564,29446,6,6.37
4,32157739,13917,1,5.36
...,...,...,...,...
54325,33443728,63825,2,10.39
54326,33499561,27741,1,9.07
54327,32895248,65895,2,1.65
54328,32792276,20134,6,8.04


In [162]:
# Let's select the variables that we want to standardize
numeric = pd.concat([df[['n_of_products', 'products_total']], df2['local_time']], axis = 1)
numeric.head()

,n_of_products,products_total,local_time
0,2,11.88,51069
1,2,5.20,42461
2,1,6.03,42833
3,6,6.37,72921
4,1,5.36,77536


In [163]:
# I'm going to use the StandardScaler instead of the RobustScaler, because as we saw, we don't have major outliers, just very different behaviours. 
scaler = StandardScaler()

In [164]:
# Let's fit the model
scaler.fit(numeric)

# Transform the data
X_s = scaler.transform(numeric)

# Convert the array into a dataframe
numeric_s = pd.DataFrame(X_s, columns = numeric.columns)
numeric_s.head(2)

,n_of_products,products_total,local_time
0,-0.335591,0.220497,-0.621691
1,-0.335591,-0.500864,-1.172203


In [165]:
df4 = df2.copy()
df4[numeric.columns] = numeric_s
df4.head()

,order_id,local_time,store_address,n_of_products,products_total,final_status,AR,BR,CI,CL,...,PE,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID
0,33446280,-0.621691,55379,-0.335591,0.220497,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33107339,-1.172203,23487,-0.335591,-0.500864,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,32960645,-1.148412,62229,-0.746959,-0.411234,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,32089564,0.775821,29446,1.309881,-0.374518,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,32157739,1.070966,13917,-0.746959,-0.483586,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Now we repeat the Balancing process with SMOTETomek

In [166]:
# Dividing in predictors and response, the variable we want to balance
y2 = df4['final_status']
X2 = df4.drop('final_status', axis=1)

In [167]:
# Initiating the method
sm = SMOTETomek(random_state=1)

# Adjusting the model
Xres2, yres2 = sm.fit_resample(X2, y2)

In [168]:
df5 = pd.concat([Xres2, yres2], axis=1)
df5.head()

,order_id,local_time,store_address,n_of_products,products_total,AR,BR,CI,CL,CR,...,PR,PT,RO,TR,UA,UY,DELAYED,NOT_PAID,PAID,final_status
0,33446280,-0.621691,55379,-0.335591,0.220497,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,33107339,-1.172203,23487,-0.335591,-0.500864,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,32960645,-1.148412,62229,-0.746959,-0.411234,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
3,32089564,0.775821,29446,1.309881,-0.374518,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,32157739,1.070966,13917,-0.746959,-0.483586,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [169]:
# Checking the balanced value counts
df5.final_status.value_counts()

1    43191
0    43191
Name: final_status, dtype: int64

In [170]:
# Last, I'm going to change the dtypes of order and store id to 'object', so that they don't affect quantitatively to the model.
df5[['order_id', 'store_address']] = df5[['order_id', 'store_address']].astype('object')
df5.dtypes

order_id           object
local_time        float64
store_address      object
n_of_products     float64
products_total    float64
AR                  uint8
BR                  uint8
CI                  uint8
CL                  uint8
CR                  uint8
DO                  uint8
EC                  uint8
EG                  uint8
ES                  uint8
FR                  uint8
GE                  uint8
GT                  uint8
IT                  uint8
KE                  uint8
MA                  uint8
PA                  uint8
PE                  uint8
PR                  uint8
PT                  uint8
RO                  uint8
TR                  uint8
UA                  uint8
UY                  uint8
DELAYED             uint8
NOT_PAID            uint8
PAID                uint8
final_status        int64
dtype: object

In [171]:
# Saving to a csv file
df5.to_csv('data/train_encoded_standardized_balanced.csv')